In [1]:
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import math
from sklearn.metrics import confusion_matrix 
from scipy import stats
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

import statsmodels.api as sm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [21]:
# read data
claim_df =  pd.read_csv("./data/ClaimLevel.csv")
policy_df = pd.read_csv("./data/PolicyLevel.csv")

"""
                        # Data Preparation & Feature generation#
Generate a single record per policy that retains max information possible
"""
claim_df['CustomerPaidAmount'] = claim_df['ClaimedAmount'] - claim_df['PaidAmount']
claim_df_grouped = claim_df.groupby(['PolicyId'])
claim_per_policy_df = claim_df_grouped[['ClaimedAmount', 'PaidAmount', 'CustomerPaidAmount']].sum()
claim_per_policy_df['ClaimsCount'] = claim_df_grouped.size()
claim_per_policy_df['AvgClaimAmount'] = claim_df_grouped[['ClaimedAmount']].mean()
claim_per_policy_df['AvgPaidAmount'] = claim_df_grouped[['PaidAmount']].mean()
claim_per_policy_df['AvgCustomerPaidAmount'] = claim_df_grouped[['CustomerPaidAmount']].mean()
claim_per_policy_df['stdClaimAmount'] = claim_df_grouped[['ClaimedAmount']].std()
claim_per_policy_df['stdPaidAmount'] = claim_df_grouped[['PaidAmount']].std()
claim_per_policy_df['stdCustomerPaidAmount'] = claim_df_grouped[['CustomerPaidAmount']].std()
claim_per_policy_df['VarianceClaimAmount'] = claim_df_grouped[['ClaimedAmount']].var()
claim_per_policy_df['VariancePaidAmount'] = claim_df_grouped[['PaidAmount']].var()
claim_per_policy_df['VarianceCustomerPaidAmount'] = claim_df_grouped[['CustomerPaidAmount']].var()
claim_per_policy_df['maxClaimAmount'] = claim_df_grouped[['ClaimedAmount']].max()
claim_per_policy_df['minClaimAmount'] = claim_df_grouped[['ClaimedAmount']].min()
claim_per_policy_df['medianClaimAmount'] = claim_df_grouped[['ClaimedAmount']].median()
claim_per_policy_df['maxCustomerPaidAmount'] = claim_df_grouped[['ClaimedAmount']].max()
claim_per_policy_df['minCustomerPaidAmount'] = claim_df_grouped[['ClaimedAmount']].min()
claim_per_policy_df['medianCustomerPaidAmount'] = claim_df_grouped[['ClaimedAmount']].median()

policy_df = policy_df.join(claim_per_policy_df, on=['PolicyId'])

# Last seen Date: CancelDate if not null else '2016-12-31' (Last date in the data set)
policy_df['LastSeenDate'] = policy_df['CancelDate']
policy_df['LastSeenDate'].fillna(pd.to_datetime('2016-12-31'), inplace=True)

# Date conversions
policy_df['EnrollDate'] = pd.to_datetime(policy_df['EnrollDate'])
policy_df['CancelDate'] = pd.to_datetime(policy_df['CancelDate'])
policy_df['LastSeenDate'] = pd.to_datetime(policy_df['LastSeenDate'])

# Length of Stay "LOS"
policy_df['LOS'] = ((policy_df['LastSeenDate'] - policy_df['EnrollDate'])/np.timedelta64(1, 'M'))
policy_df = policy_df.round(decimals=2)

# Insurance Status: Active means 1 else 0
policy_df['InsuranceStatus'] = policy_df['CancelDate'].apply(lambda x: 0 if pd.notnull(x) else 1)

# drop null values
policy_df.dropna(axis='index', subset=['MonthlyPremium', 'ClaimedAmount', 'PaidAmount'], inplace=True)

# If no claims were made then all amounts are equal to 0 instead of NULL
for column in policy_df.columns:
    policy_df[column].fillna(0, inplace=True)

# display(claim_df.head())
display(policy_df.head(6))

,PolicyId,EnrollDate,CancelDate,MonthlyPremium,ClaimedAmount,PaidAmount,CustomerPaidAmount,ClaimsCount,AvgClaimAmount,AvgPaidAmount,...,VarianceCustomerPaidAmount,maxClaimAmount,minClaimAmount,medianClaimAmount,maxCustomerPaidAmount,minCustomerPaidAmount,medianCustomerPaidAmount,LastSeenDate,LOS,InsuranceStatus
2,93090,2010-11-11,1970-01-01,23.53,762.49,329.46,433.03,3.0,254.16,109.82,...,7971.91,510.48,100.00,152.01,510.48,100.00,152.01,2016-12-31,73.66,1
5,93258,2010-11-11,2016-05-24,21.66,33.99,30.60,3.39,3.0,11.33,10.20,...,0.00,11.33,11.33,11.33,11.33,11.33,11.33,2016-05-24,66.40,0
7,1324,2009-05-11,1970-01-01,38.84,754.76,315.10,439.66,7.0,107.82,45.01,...,3215.09,363.81,15.23,89.88,363.81,15.23,89.88,2016-12-31,91.70,1
9,1406,2009-02-20,1970-01-01,41.44,1518.46,1095.27,423.19,8.0,189.81,136.91,...,2545.23,777.98,30.95,97.62,777.98,30.95,97.62,2016-12-31,94.33,1
10,1413,2009-02-23,1970-01-01,21.24,336.81,0.00,336.81,5.0,67.36,0.00,...,1740.78,134.76,35.06,46.23,134.76,35.06,46.23,2016-12-31,94.23,1
13,1499,2009-05-29,1970-01-01,20.53,1801.90,1355.14,446.76,2.0,900.95,677.57,...,47345.80,1732.38,69.52,900.95,1732.38,69.52,900.95,2016-12-31,91.11,1


In [22]:
"""
                   # Data Description #
Churn --> 'C' 
Non-Churn --> 'NC'
"""
total_policies = policy_df.shape[0]
curr_NC = policy_df[policy_df['InsuranceStatus']==1].shape[0]/float(total_policies)
curr_C = policy_df[policy_df['InsuranceStatus']==0].shape[0]/float(total_policies)
print("Total polices", total_policies)
print("NC:C = ",curr_NC, ":", curr_C)

Total polices 31526
NC:C =  0.8917401509864873 : 0.10825984901351265


In [23]:
# Sampling Data
churn_indices = policy_df[policy_df['InsuranceStatus']==0].index.values
nonchurn_indices = policy_df[policy_df['InsuranceStatus']==1].index.values
# print(len(churn_indices), len(nonchurn_indices))

def sample_train_data(num_samples=None, churn_percent=None):
    """
    :param num_samples: total number of samples to return
    :param nc: ratio of non-churn in the return data set
    :param c: ration of churn in the return data set
    """
    # check to see if nc + c = 1
    churn_sample_space = churn_indices
    non_churn_sample_space = nonchurn_indices
        
    if num_samples is None and churn_percent is None:
        # use max samples possible (only 80% for train)
        churn_percent = curr_C 
        num_samples = int(len(churn_indices)*0.8/churn_percent)
    elif num_samples is None:
        num_samples = int(len(churn_indices)*0.8/churn_percent)
    
    if churn_percent==None:
        # default rate
        churn_percent = curr_C 
    elif churn_percent > 1:
        raise ValueError('Churn percent cannot be > 1')
    
    non_churn_percent = 1 - churn_percent
    nc_samples_count = int(num_samples * non_churn_percent)
    c_samples_count = int(num_samples * churn_percent)
    
    
    # prune train indices if looking for test set
    churn_sample_indices = np.random.choice(churn_sample_space, size=c_samples_count, replace=False)
    nonchurn_sample_indices = np.random.choice(non_churn_sample_space, size=nc_samples_count, replace=False)
    sample_indices = np.concatenate((churn_sample_indices, nonchurn_sample_indices))
    # shuffle
    np.random.shuffle(sample_indices)
    sample = policy_df.loc[sample_indices, :]
    return sample

def sample_test_data(skip_indices=[]):
    
    churn_sample_space = np.setdiff1d(churn_indices, skip_indices)
    non_churn_sample_space = np.setdiff1d(nonchurn_indices, skip_indices)
    churn_sample_indices = churn_sample_space
    nonchurn_sample_indices = np.random.choice(non_churn_sample_space, size=len(churn_sample_indices), replace=False)
    sample_indices = np.concatenate((churn_sample_indices, nonchurn_sample_indices))
    # shuffle
    np.random.shuffle(sample_indices)
    sample = policy_df.loc[sample_indices, :]
    return sample
    
# test case
test = True
if test == False:
#     sample = sample_data(100, 0.8, 0.2)
    sample = sample_data()
    sample.head()
    sample_total_policies = sample.shape[0]
    NC = sample[sample['InsuranceStatus']==1].shape[0]/float(sample_total_policies)
    C = sample[sample['InsuranceStatus']==0].shape[0]/float(sample_total_policies)
    print("Total polices", total_policies)
    print("NC:C = ",NC, ":", C)

In [24]:
# data set 1: contains NC and C in the same ratio as the given in the original data and num_samples = 0.8%
train_set1_df = sample_train_data()
test_set1_df = sample_test_data(skip_indices=train_set1_df.index.values)

# # data set 2: NC:C = 1:1 and contains max samples possible
train_set2_df = sample_train_data(churn_percent=0.5)
test_set2_df = sample_test_data(skip_indices=train_set2_df.index.values)


# # data set 3: NC:C = 1:2 and contains max samples possible
train_set3_df = sample_train_data(churn_percent=0.66)
test_set3_df = sample_test_data(skip_indices=train_set3_df.index.values)

# # data set 4: NC:C = 2:1 and contains max samples possible
train_set4_df = sample_train_data(churn_percent=0.33)
test_set4_df = sample_test_data(skip_indices=train_set4_df.index.values)

# # data set 5: NC:C = 4:1 and contains max samples possible
train_set5_df = sample_train_data(churn_percent=0.20)
test_set5_df = sample_test_data(skip_indices=train_set5_df.index.values)

In [25]:
select_columns = list(
    set(policy_df.columns) - set(['PolicyId', 'EnrollDate', 'CancelDate', 'InsuranceStatus', 'LastSeenDate',
#                                  'ClaimedAmount', 'PaidAmount', 'CustomerPaidAmount' , 
#                                  'AvgClaimAmount', 'AvgPaidAmount', 'stdClaimAmount', 'stdPaidAmount'
#                                   'minClaimAmount', 'maxClaimAmount', 'medianClaimAmount'
                                 ]))

datasets  = [
        [train_set1_df, test_set1_df],
        [train_set2_df, test_set2_df],
        [train_set3_df, test_set3_df],
        [train_set4_df, test_set4_df],
        [train_set5_df, test_set5_df]
        ]

In [26]:
"""
                                        # RF Model #
"""
from sklearn.ensemble import RandomForestClassifier as RF

i = 1
for dataset in datasets:
    print("train Data set :", i)
    i += 1
    X_train = dataset[0][select_columns]
    X_test = dataset[1][select_columns]
    y_train = dataset[0][['InsuranceStatus']]
    y_test = dataset[1][['InsuranceStatus']]
    X_train, X_test = scaler.fit_transform(X_train), scaler.fit_transform(X_test)
    for weight in [1]:
        model = RF(n_estimators=100, class_weight={0:2,1:1})
        model.fit(X_train, y_train)
        y_predicted = model.predict(X_test)
        print("\t Weight ", weight)
#         print("\t Train",model.score(X_train, y_train))
        print("\t Test accuracy",model.score(X_test, y_test))
#         print(confusion_matrix(y_test, y_predicted))
#         X2 = sm.add_constant(X_train)
#         est = sm.Logit(y_train, X2)
#         est2 = est.fit()
#         print(est2.summary())

train Data set : 1
	 Weight  1
	 Test accuracy 0.500732064422
train Data set : 2
	 Weight  1
	 Test accuracy 0.588579795022
train Data set : 3
	 Weight  1
	 Test accuracy 0.550438596491
train Data set : 4
	 Weight  1
	 Test accuracy 0.538067349927
train Data set : 5
	 Weight  1
	 Test accuracy 0.505856515373


In [28]:
"""
                                        # Logistic Model #
"""
from sklearn.linear_model import LogisticRegression

i = 1
for dataset in datasets:
    print("train Data set :", i)
    i += 1
    X_train = dataset[0][select_columns]
    X_test = dataset[1][select_columns]
    y_train = dataset[0][['InsuranceStatus']]
    y_test = dataset[1][['InsuranceStatus']]
    X_train, X_test = scaler.fit_transform(X_train), scaler.fit_transform(X_test)
    for weight in [0.5, 0.75, 1, 1.5, 2, 3, 4]:
        point_weights = dataset[0].InsuranceStatus.apply(lambda x: weight if x == 0 else 1)
        model = LogisticRegression(penalty='l1',)
        model.fit(X_train, y_train, point_weights)
        y_predicted = model.predict(X_test)
        print("\t Weight on disenrolled polcies", weight)
        print("\t Test Accuracy",model.score(X_test, y_test))
        print("",confusion_matrix(y_test, y_predicted))

train Data set : 1
	 Weight on disenrolled polcies 0.5
	 Test Accuracy 0.5
	 Weight on disenrolled polcies 0.75
	 Test Accuracy 0.500732064422
	 Weight on disenrolled polcies 1
	 Test Accuracy 0.501464128843
	 Weight on disenrolled polcies 1.5
	 Test Accuracy 0.501464128843
	 Weight on disenrolled polcies 2
	 Test Accuracy 0.501464128843
	 Weight on disenrolled polcies 3
	 Test Accuracy 0.502928257687
	 Weight on disenrolled polcies 4
	 Test Accuracy 0.505124450952
train Data set : 2
	 Weight on disenrolled polcies 0.5
	 Test Accuracy 0.507320644217
	 Weight on disenrolled polcies 0.75
	 Test Accuracy 0.53953147877
	 Weight on disenrolled polcies 1
	 Test Accuracy 0.576134699854
	 Weight on disenrolled polcies 1.5
	 Test Accuracy 0.500732064422
	 Weight on disenrolled polcies 2
	 Test Accuracy 0.494875549048
	 Weight on disenrolled polcies 3
	 Test Accuracy 0.5
	 Weight on disenrolled polcies 4
	 Test Accuracy 0.5
train Data set : 3
	 Weight on disenrolled polcies 0.5
	 Test Accuracy 0

In [29]:
"""
                                        # neural nets Model #
"""
from sklearn.neural_network import MLPClassifier

datasets  = [
        [train_set1_df, test_set1_df],
        [train_set2_df, test_set2_df],
        [train_set3_df, test_set3_df],
        [train_set4_df, test_set4_df]
        ]

i = 1
for dataset in datasets:
    print("train Data set :", i)
    i += 1
    X_train = dataset[0][select_columns]
    X_test = dataset[1][select_columns]
    y_train = dataset[0][['InsuranceStatus']]
    y_test = dataset[1][['InsuranceStatus']]
    X_train, X_test = scaler.fit_transform(X_train), scaler.fit_transform(X_test)
    model = MLPClassifier(solver='lbfgs', alpha=1e-4, hidden_layer_sizes=(16, 32, 24, 8), random_state=1, 
                         activation='relu')
    model.fit(X_train, y_train)
    y_predicted = model.predict(X_test)
#         print("\t Train",model.score(X_train, y_train))
    print("\t Test",model.score(X_test, y_test))
#     print(confusion_matrix(y_test, y_predicted))

train Data set : 1
	 Test 0.502928257687
train Data set : 2
	 Test 0.599560761347
train Data set : 3
	 Test 0.523391812865
train Data set : 4
	 Test 0.568814055637


In [14]:
claim_df['PolicyId'].nunique()

31526